In [4]:
import os
import librosa
import soundfile as sf
from glob import glob
from datetime import datetime
import time


def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Failed to create the directory")

#---------------------------------------------------------wav파일 처리---------------------------------------------------------
def trim_audio(audio, i, sec_first_time, sec_last_time):
    #print("Present workig directory : " + os.getcwd())
    sr = 16000 #samplimg rate
        
    y, sr = librosa.load(audio, sr=sr)
    ny = y[int(sr * sec_first_time) : int(sr * sec_last_time)]
    
    #save_file = audio[:-4] + "_" + str(sec_first_time)
    #save_file = save_path + '/' + audio_name[:-4] + "_" + str(count)
    
    #파일 이름과 경로 알아내기
    audio_name = os.path.basename(audio)
    file_directory = os.path.dirname(audio)
    parent_directory = os.path.dirname(file_directory)
    
    #파일 저장
    save_directory = parent_directory + '/' + audio_name[:-4]
    create_directory(save_directory)
    save_file = save_directory +'/' + audio_name[:-4] + '_' + str(i) + '.wav'
    sf.write(save_file, ny, sr)
    
    return save_file
    

#---------------------------------------------------------txt파일 처리---------------------------------------------------------
def trim_time(time):
    #시작 시간 (HH:MM:SS)
    first_time = time.replace('\n', '').replace(',','.').replace(' ','').split('-->')[0]

    #끝나는 시간 (HH:MM:SS)
    last_time = time.replace('\n', '').replace(',','.').replace(' ','').split('-->')[1]
    
    #시작시간(first_time)과 끝나는시간(last_time)을 초단위로 표현
    x = datetime.strptime(first_time,"%H:%M:%S.%f")
    y = datetime.strptime(last_time,"%H:%M:%S.%f")
    
    sec_first_time = (x.hour * 3600) + (x.minute * 60) + (x.second) + (x.microsecond / 1000000) + 0.1
    sec_last_time = (y.hour * 3600) + (y.minute * 60) + (y.second) + (y.microsecond / 1000000) + 0.1
    
#     print('시작시간 : ', sec_first_time)
#     print('끝나는시간 : ', sec_last_time)
    return sec_first_time, sec_last_time

#---------------------------------------------------------main---------------------------------------------------------

base_path = 'C:/Users/hoy26/OneDrive/바탕 화면/audit'
#base_path = 'C:/Users/USER/Desktop/전사 파일 작업 완료/2022_07_21_Vrew_완료_한승연/2022_07_21_Vrew_완료_한승연'
os.chdir(base_path) # base_path로 작업 디렉토리 이동

audio_path = base_path + '/audio'
text_path = base_path + '/text'


#audio_list = os.listdir(audio_path)
audio_list = glob(audio_path +'/*.wav') #경로가 포함된 .wav파일 list
text_list = glob(text_path + '/*.txt') #경로가 포함된 .txt파일 list
# print(audio_list)
# print(text_list)

transcript_list = []
fname_list = []
audio_data_list = []
data_list = []


for audio, text in zip(audio_list, text_list): 
    data_dic = {}
    print("audio : " + audio)
    print("text : " + text)
    print("="*110)
    
    with open(text, 'r', encoding='utf-8') as f:
        text_info = f.readlines()
        # 구간 인덱스 : 1, 5, 9 4씩
        # 전사 인덱스 : 2, 6, 10 4씩
        time_list = text_info[1::4]
        label_list = text_info[2::4]
        
        for i, (time, label) in enumerate(zip(time_list, label_list)):
            audio_data_dic = {}
            
            sec_first_time, sec_last_time = trim_time(time)
            audio_data_dic["transcript"] = label.replace('\n','')
            
            save_file = trim_audio(audio, i, sec_first_time, sec_last_time)

            #파일 이름 slice
            sl_save_file = save_file.split('/')
            #print(sl_save_file)
            file_name = '/'.join(sl_save_file[-3:])
            #audio_data_dic["fname"] = file_name
            
            duration = sec_last_time - sec_first_time
            #files 키 추가
            audio_data_dic["files"] = [{"channels" : 1, "sample_rate" : 16000.0,
                                        "bitrate" : 256000.0, "duration" : duration ,
                                       "num_samples" : duration * 16000, "encoding" : '"Signed Integer PCM"',
                                       "silent" : False, "fname" : file_name, "speed" : 1}]
            audio_data_dic["original_duration"] = 8.539
            audio_data_dic["original_num_samples"] = 136624
            audio_data_list.append(audio_data_dic)
    
    data_dic["data"] = audio_data_list
#print(audio_data_list)

audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\고석환_S0001.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\고석환_S0001.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\고석환_S0002.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\고석환_S0002.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\고수정_S0003.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\고수정_S0003.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\고수정_S0004.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\고수정_S0004.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\고수정_S0005.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\고수정_S0005.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\박소진_S0006.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\박소진_S0006.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\한승연_S0007.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\한승연_S0007.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/audit/audio\한승연_S0008.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/audit/text\한승연_S0

In [5]:
create_directory(base_path + '/result')
os.chdir(base_path + '/result')
print("Directory to save : " + os.getcwd())

#결과를 json파일로 저장
import json

with open("trim_audio_file.json", "wt", encoding='utf-8-sig') as json_file:
    json.dump(data_dic, json_file, ensure_ascii=False, indent = 4)
print("저장완료!")

Directory to save : C:\Users\hoy26\OneDrive\바탕 화면\audit\result
저장완료!
